<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M3/exercises/Claim_Similarity_and_Classification_SBERT_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install sentence_transformers

In [7]:
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd

In [8]:
df_row = pd.read_csv('https://github.com/SDS-AAU/SDS-master/raw/master/00_data/patent_claim5000.csv')

In [9]:
df_row.head()

,patent_id,claim_number,text,sequence,dependent,exemplary,subclass
0,10606734,6,6. A computer-program product for intelligent ...,5,NaN,0,A
1,10606734,7,"7. The computer program product of claim 6 , w...",6,claim 6,0,A
2,10606734,8,"8. The computer program product of claim 6 , f...",7,claim 6,0,A
3,10606734,9,"9. The computer program product of claim 8 , w...",8,claim 8,0,A
4,10606734,10,"10. The computer program product of claim 6 , ...",9,claim 6,0,A


In [10]:
df_row.iloc[0]['text']

'6. A computer-program product for intelligent mobile device selection for mobile application testing, the computer-program product comprising: a computer-readable storage medium readable by a processing circuit and storing instructions for execution by the processing circuit for performing a method comprising: determining features of a new mobile application to be tested; comparing, by a processor, the features of the new mobile application to be tested with features of multiple known mobile applications to identify one or more known mobile applications with similar features; based at least in part on automated analysis of user reviews of the one or more known mobile applications operating in one or more types of mobile devices, providing one or more risk scores for operation of the new mobile application in the one or more types of mobile devices, wherein providing the one or more risk scores comprises generating a respective risk score for operating the new mobile application in eac

In [12]:
df_newmergetext = df_row.groupby('patent_id')['text'].apply(lambda x: "{%s}" % ', '.join(x))
# type(df_newmergetext)

In [13]:
df_newmergetext.head()

patent_id
10606734    {6. A computer-program product for intelligent...
10606736    {1. A computer-implemented method for creation...
10606737    {1. A method for testing a resource constraine...
10606738    {1. A method, comprising: receiving results fr...
10606739    {1. A device, comprising: a memory; and one or...
Name: text, dtype: object

In [14]:
df_newmergetext = df_newmergetext.to_frame()
# df_newmergetext.head()
# df_newmergetext.iloc[0]['text']

In [15]:
df = pd.merge(df_row.drop_duplicates(subset='patent_id'), df_newmergetext, on='patent_id', how='inner')

In [16]:
df.head()

,patent_id,claim_number,text_x,sequence,dependent,exemplary,subclass,text_y
0,10606734,6,6. A computer-program product for intelligent ...,5,NaN,0,A,{6. A computer-program product for intelligent...
1,10606736,1,1. A computer-implemented method for creation ...,0,NaN,1,A,{1. A computer-implemented method for creation...
2,10606737,1,1. A method for testing a resource constrained...,0,NaN,1,A,{1. A method for testing a resource constraine...
3,10606738,1,"1. A method, comprising: receiving results fro...",0,NaN,1,A,"{1. A method, comprising: receiving results fr..."
4,10606739,1,"1. A device, comprising: a memory; and one or ...",0,NaN,1,A,"{1. A device, comprising: a memory; and one or..."


In [17]:
df['text_y'] = df['text_y'].str.strip('{}')

In [18]:
df = df.drop('text_x', axis=1)

In [19]:
df.head()

,patent_id,claim_number,sequence,dependent,exemplary,subclass,text_y
0,10606734,6,5,NaN,0,A,6. A computer-program product for intelligent ...
1,10606736,1,0,NaN,1,A,1. A computer-implemented method for creation ...
2,10606737,1,0,NaN,1,A,1. A method for testing a resource constrained...
3,10606738,1,0,NaN,1,A,"1. A method, comprising: receiving results fro..."
4,10606739,1,0,NaN,1,A,"1. A device, comprising: a memory; and one or ..."


In [20]:
df = df.rename(columns={"text_y": "text"}, errors="raise")

In [21]:
total_patents = df.count()
total_patents

patent_id       282
claim_number    282
sequence        282
dependent         0
exemplary       282
subclass        282
text            282
dtype: int64

In [ ]:
## Torch tryout
# df_test_tensor = df.drop(['text', 'dependent'], axis=1)
# df_test_tensor
# sorted, indices = torch.sort(df_tensor, 0)

In [25]:
claims = list(df.text)
patent_id = list(df.patent_id)

In [28]:
from sentence_transformers import SentenceTransformer

In [30]:
#embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

100%|██████████| 461M/461M [00:19<00:00, 24.2MB/s]


In [ ]:
#Test_claim_embeddings = embedder.encode(claims)

In [32]:
def get_top_n_similar_patents_df(new_claim, claims):
    # embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
    embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
    query_embeddings = embedder.encode([new_claim])

    # list of patent claims
    claim_embeddings = embedder.encode(claims)

    # get top 100 patent claims based on cosine similarity
    closest_n = 100
    distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    # save similar patents info
    top_claim_ids = []
    top_claims = []
    top_similarity_scores = []


    print('New_claim: ' + new_claim + '\n')

    # Find the closest 100 patent claims for each query new_claim based on cosine similarity
    for idx, distance in results[0:closest_n]:
        top_claim_ids.append(patent_id[idx])
        top_claims.append(claims[idx])
        top_similarity_scores.append(round((1-distance), 4))
        print('Patent ID: ' + str(patent_id[idx]))
        print('PubMed Claim: ' + claims[idx])
        print('Similarity Score: ' + "%.4f" % (1-distance))
        print('\n')
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
        'claims': top_claims,
    })
    
    return top_100_similar_patents_df

In [33]:
new_claim = ['The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.']

In [34]:
# search for similar patent claim

if os.path.exists('top_100_similar_patents_df.csv'):
    get_top_n_similar_patents_df = pd.read_csv('top_100_similar_patents_df.csv')

else:
    get_top_n_similar_patents_df = get_top_n_similar_patents_df(new_claim[0], claims)

New_claim: The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.

Patent ID: 10606917
PubMed Claim: 1. An alternating least square recommendation system, comprising a processor; and a memory, the memory storing instructions to cause the processor to perform: partially updating a user's feature by evaluating an update ratio and a predefined threshold ratio, the pre-defined threshold ratio being configur

In [ ]:
# type(get_top_n_similar_patents_df)

In [35]:
get_top_n_similar_patents_df.head()

,top_claim_ids,cosine_similarity,claims
0,10606917,0.6802,1. An alternating least square recommendation ...
1,10606970,0.6652,1. A computer-implemented method for statistic...
2,10606918,0.6614,"1. A convolution engine, comprising: an input ..."
3,10606971,0.6549,"1. A system, comprising a processor to: receiv..."
4,10606931,0.6529,"1. A system, comprising: one or more storage d..."


In [36]:
result = pd.concat([get_top_n_similar_patents_df, df], axis=1, join='inner')

In [37]:
result.head()

,top_claim_ids,cosine_similarity,claims,patent_id,claim_number,sequence,dependent,exemplary,subclass,text
0,10606917,0.6802,1. An alternating least square recommendation ...,10606734,6,5,NaN,0,A,6. A computer-program product for intelligent ...
1,10606970,0.6652,1. A computer-implemented method for statistic...,10606736,1,0,NaN,1,A,1. A computer-implemented method for creation ...
2,10606918,0.6614,"1. A convolution engine, comprising: an input ...",10606737,1,0,NaN,1,A,1. A method for testing a resource constrained...
3,10606971,0.6549,"1. A system, comprising a processor to: receiv...",10606738,1,0,NaN,1,A,"1. A method, comprising: receiving results fro..."
4,10606931,0.6529,"1. A system, comprising: one or more storage d...",10606739,1,0,NaN,1,A,"1. A device, comprising: a memory; and one or ..."


In [38]:
print(result.count())

top_claim_ids        100
cosine_similarity    100
claims               100
patent_id            100
claim_number         100
sequence             100
dependent              0
exemplary            100
subclass             100
text                 100
dtype: int64


In [ ]:
# import pandas as pd
# import torch

# # determine the supported device
# def get_device():
#     if torch.cuda.is_available():
#         device = torch.device('cuda:0')
#     else:
#         device = torch.device('cpu') # don't have GPU 
#     return device

# # convert a df to tensor to be used in pytorch
# def df_to_tensor(df):
#     device = get_device()
#     return torch.from_numpy(df.values).float().to(device)

# df_tensor = df_to_tensor(result[['cosine_similarity','claim_number','patent_id']])
# # series_tensor = df_to_tensor(series)

In [ ]:
# df_tensor
# sorted, indices = torch.sort(df_tensor, 0)
# a = torch.topk(df_tensor, 2, dim = 0)
# a

In [39]:
k_similar_patents = result.nlargest(10, ['cosine_similarity'])

In [40]:
k_similar_patents

,top_claim_ids,cosine_similarity,claims,patent_id,claim_number,sequence,dependent,exemplary,subclass,text
0,10606917,0.6802,1. An alternating least square recommendation ...,10606734,6,5,NaN,0,A,6. A computer-program product for intelligent ...
1,10606970,0.6652,1. A computer-implemented method for statistic...,10606736,1,0,NaN,1,A,1. A computer-implemented method for creation ...
2,10606918,0.6614,"1. A convolution engine, comprising: an input ...",10606737,1,0,NaN,1,A,1. A method for testing a resource constrained...
3,10606971,0.6549,"1. A system, comprising a processor to: receiv...",10606738,1,0,NaN,1,A,"1. A method, comprising: receiving results fro..."
4,10606931,0.6529,"1. A system, comprising: one or more storage d...",10606739,1,0,NaN,1,A,"1. A device, comprising: a memory; and one or ..."
5,10606981,0.6499,1. A computer-implemented method for space fra...,10606740,1,0,NaN,1,A,1. A system comprising: a processor comprising...
6,10606909,0.6270,1. A computer-implemented method for optimizin...,10606741,1,0,NaN,1,A,1. A process performed by a computing device f...
7,10606856,0.6263,"1. A method, comprising: ingesting, by a data ...",10606742,1,0,NaN,1,A,1. A method for programming a motor drive havi...
8,10606749,0.6246,1. A non-transitory computer-readable storage ...,10606743,1,0,NaN,1,A,"1. An apparatus, comprising: an array of non-v..."
9,10606819,0.6236,1. A method of reducing a number of event reco...,10606744,1,0,NaN,1,A,1. A method for accessing a flash memory modul...


In [41]:
final_result = k_similar_patents.mode()
print('Prediction for Subclass of New Patent is: ', final_result.iloc[0]['subclass'])

Prediction for Subclass of New Patent is:  A
